#### Imports

In [1]:
import numpy as np
import tensorflow as tf
import emoji
import pandas as pd
import json
import pickle


from keras_preprocessing.text import Tokenizer
from keras.models import load_model, model_from_json
from keras_preprocessing.sequence import pad_sequences

np.random.seed(0)


2024-12-01 06:00:51.739555: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 06:00:51.743013: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 06:00:51.798404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 06:00:51.798453: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 06:00:51.798499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

#### Add model name

In [2]:
model_folder = "models/training set v.3.0"

model_name = "epochs_100_layers_55_50_45_40_35_30_25_20_10"
# model_name = "epochs_15_layers_32_64_128_256"

model_json_path = f"{model_folder}/json/{model_name}.json"

model_weight_path = f"{model_folder}/weights/{model_name}.h5"


#### Loading the model

In [3]:

json_file = open(f"{model_json_path}", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(model_weight_path)
print("Loaded model from disk")

2024-12-01 06:00:59.985870: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-01 06:00:59.988244: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loaded model from disk


#### Emoji Dictionary

In [4]:
emoji_dictionary = {
    0: ":red_heart:",  # Love # X
    1: ":face_with_tears_of_joy:",  # Laughter X
    2: ":grinning_face_with_big_eyes:", # Happiness # X
    3: ":loudly_crying_face:",  # Sadness # X
    4: ":smiling_face_with_heart-eyes:",  # Adoration X
    5: ":fire:",  # Excitement X
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude X
    8: ":angry_face:",  # Anger X
    9: ":thinking_face:",  # Contemplation X
}

#### Function to get emoji from class

In [5]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [6]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

#### Add own data

In [7]:
test = [
    "I am seeing what is happening",
    "Does loading and unloading even work?",
    "Naniii",
    "Wtf is happening",
    "Im dying",
    "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA",
    "I like your hoodie",
    "Thank you for the help",
    "Your jokes are so funny",
    "I made it",
    "You are so hot",
    "I love you so much",
    "Good job!",
    "I am trying", "I want to cry", "This is just sad",
    "I love sleeeping :)",
    "Wait what is the syllabus even rip",
    "slayyy",
    "FIRE",
    "If you're happy and you know it clap your hands",
    "Less epochs is better huh?",
    "Confusion matrices make it less confusing",
    "Zero errors on first run are scary",
    "THIS FINALLY WORKS MWAHAHAHA",
    "Hapi hapi hapi",
    "I love it when the code runs",
]


test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=loaded_model.layers[0].input_shape[1], padding='post', truncating='post')

#### See Results (emoji classes view)

In [8]:
y_pred = loaded_model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)
y_pred

1/1 [==============================] - 4s 4s/step


array([9, 9, 9, 8, 3, 9, 9, 7, 1, 2, 5, 0, 1, 3, 3, 3, 0, 7, 9, 5, 4, 7,
       2, 9, 9, 9, 0])

#### See results (actual emoji view)

In [9]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am seeing what is happening 🤔
Does loading and unloading even work? 🤔
Naniii 🤔
Wtf is happening 😠
Im dying 😭
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 🤔
I like your hoodie 🤔
Thank you for the help 🙏
Your jokes are so funny 😂
I made it 😃
You are so hot 🔥
I love you so much ❤️
Good job! 😂
I am trying 😭
I want to cry 😭
This is just sad 😭
I love sleeeping :) ❤️
Wait what is the syllabus even rip 🙏
slayyy 🤔
FIRE 🔥
If you're happy and you know it clap your hands 😍
Less epochs is better huh? 🙏
Confusion matrices make it less confusing 😃
Zero errors on first run are scary 🤔
THIS FINALLY WORKS MWAHAHAHA 🤔
Hapi hapi hapi 🤔
I love it when the code runs ❤️


#### Accuracy 

##### If you added test data and want to see accuracy metrics, follow the emoji dictionary and add classes in the labels array

In [74]:
# Calculate accuracy
# These values are subjective
test_labels = [9,3,8,8,3,3,4,7,1,2,5,0,6,3,3,3,0,7,5,5,2,7,9,3,5,2,0]

correct_predictions = np.sum(y_pred == test_labels)
total_predictions = len(test_labels)
accuracy = correct_predictions / total_predictions


# Calculate loss
# Assuming you have a loss function defined or you're using a standard loss function like categorical crossentropy
loss = 0.0
for i in range(len(test_labels)):
    true_label = test_labels[i]
    predicted_value = y_pred[i]  
    loss += (true_label - predicted_value) ** 2
loss /= len(test_labels)
print(f'Test loss (MSE): {loss}')
print(f'Test accuracy: {accuracy}')

Test loss (MSE): 10.851851851851851
Test accuracy: 0.5925925925925926


#### Accuracy represented in an easy to understand visual format

In [75]:
from termcolor import colored

data = [(test[i], label_to_emoji(y_pred[i]), label_to_emoji(test_labels[i])) for i in range(len(test))]

# Define headers
headers = ['Test Sentence', 'Predicted Emoji', 'True Emoji']

# Display the headers

print(colored(f"{headers[0]:<70} | {headers[1]:<1} | {headers[2]}", 'yellow'))
print(colored('-' * 71 + '|' + '-' * 17 + '|' + '-' * (len(headers[2])+1), 'yellow'))

# Display the data
for i in range(len(data)):
    if data[i][1] == data[i][2]:
        print(colored(f"{data[i][0]:<70} | {data[i][1]}\t\t | {data[i][2]}", 'green'))
    else:
        print(colored(f"{data[i][0]:<70} | {data[i][1]}\t\t | {data[i][2]}", 'red'))

    


Test Sentence                                                          | Predicted Emoji | True Emoji
-----------------------------------------------------------------------|-----------------|-----------
I am seeing what is happening                                          | 🤔		 | 🤔
Does loading and unloading even work?                                  | 🤔		 | 😭
Naniii                                                                 | 🤔		 | 😠
Wtf is happening                                                       | 😠		 | 😠
Im dying                                                               | 😭		 | 😭
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA        | 🤔		 | 😭
I like your hoodie                                                     | 🤔		 | 😍
Thank you for the help                                                 | 🙏		 | 🙏
Your jokes are so funny                                                | 😂		 | 😂
I made it                                                          